In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pipeline as p
import globals
import pandas as pd
import os

# Accompanying paper: please click [[here]](https://doi.org/10.5194/amt-14-37-2021).
### Please see the last section of this notebook (click [[here]](#Final-Results)) for final results and reproducibility checks

## Experiment 1: calibration results on the original datasets
Please note that only 2 of the 12 datasets used in the accompanying paper, namely DD1(Jun) and DD1(Oct), are provided as a part of this repository in the directory [data/](data/). Only those results that rely only on these two datasets can be reproduced.

In [3]:
# Initialising global variables. For details, please refer to globals.py
g1 = globals.My_Globals()

### Load a dataset and perform preprocessing
**Note**: The calibration algorithms rely on train-test splits to obtain an unbiased estimate of the performance of various calibration algorithms. If you wish to reproduce the results in the paper, please use the train-test splits supplied for the 2 datasets DD1(Jun) and DD1(Oct) in the directory [perm/](perm/).

To do so, please use ```new_permutations = False``` below. Using ```new_permutations = True``` instead, allows fresh train-test splits to be created which will then overwrite the supplied train-test splits. However, minor differences are to be expected in the results if fresh splits are used to conduct the experiments.

In [4]:
datasets, datasets_norm = p.load_data( g1, new_permutations = False )

In [5]:
# Displaying some top rows of the training data for the dataset DD1(Jun) for split 0  
datasets_norm[0]['DD1(Jun).csv']['train'].head()

,Ref. O3(ppb),Ref. NO2(ppb),Temp(C),RH(%),no2op1(mV),no2op2(mV),o3op1(mV),o3op2(mV),no2op1 - no2op2(mV),o3op1 - o3op2(mV)
2340,22.223,16.407,0.239336,-1.982867,1.415799,1.372104,1.322321,1.403552,0.809485,0.150671
19549,19.247,11.387,-0.008013,0.536193,0.274850,0.302993,0.143411,0.306001,-0.133636,-0.852900
57433,32.430,3.940,-1.079855,0.536193,-1.681063,-1.682500,-1.602569,-1.663134,-0.537830,-0.396731
43586,13.573,21.040,-1.052372,0.536193,-0.752005,-0.800059,-0.871346,-0.807690,0.135828,-0.761666
43299,34.350,2.677,-0.667608,0.536193,-1.094289,-1.139459,-1.035498,-1.130499,0.001096,0.059438


### Train models for various calibration algorithms

In [6]:
algoList = [ 'LS', 'LS(MIN)', 'LASSO', 'RT', 'KRR', 'NYS', 'NW(ML)', 'KNN', 'KNN-D', 'KNN(ML)', 'KNN-D(ML)' ]

p.doExperiment( g1, algoList, datasets, datasets_norm )


--------------------------------------------------------------
Training for split: 0

Dataset: DD1(Jun).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

Dataset: DD1(Oct).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

--------------------------------------------------------------
Training for split: 1

Dataset: DD1(Jun).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

Dataset: DD1(Oct).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

--------------------------------------------------------------
Training for split: 2

Dataset: DD1(Jun).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

Dataset: DD1(Oct).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

--------------------------------------------------------------
Training for split: 3

Dataset: DD1(Jun).csv
LS LS(MIN) LASSO RT KRR NYS NW(ML) KNN KNN-D KNN(ML) KNN-D(ML) 

Dataset: DD1(Oct).csv
LS LS(MIN) LASSO RT KRR 

### Calculate MAE, RMSE, MAPE(%), R2 values

In [7]:
p.gen_errorMetrics( g1, algoList, datasets, datasets_norm )

# Generating mean and std of error metrics over all the splits
p.gen_avgErrorMetrics( g1, algoList, datasets, datasets_norm )

In [8]:
# Results for KNN-D(ML) for the DD1(Jun) and DD1(Oct) datasets corresponding to Table 4 in the accompanying paper
algo = 'KNN-D(ML)'

res_o3 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'o3' + '.csv', index_col = 0 )
res_no2 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'no2' + '.csv', index_col = 0 )

res_t4 = pd.merge(res_o3, res_no2, left_index = True, right_index = True, suffixes=('_O3', '_NO2') )

### Calculate win matrices and rank all algorithms for each dataset
Please note that the ranking and win-matrix results included in this notebook cannot reproduce those in the accompanying paper (Table 3 and Table 5). This is because results in the paper are averaged over 12 datasets whereas those in the notebook use only 2 of those datasets, namely DD1(Jun) and DD1(Oct), which are provided as a part of this repository in the directory [data/](data/).

In [9]:
p.gen_winMatrix( g1, algoList, datasets, datasets_norm )

In [10]:
# Win matrix corresponding to table 3 in the paper (although does not reproduce those results)
winMatrix_o3 = pd.read_csv( g1.winMatrixPath + 'o3' + '.csv', index_col = 0 )
winMatrix_no2 = pd.read_csv( g1.winMatrixPath + 'no2' + '.csv', index_col = 0 )

### Experiments with Alphasense calibration models
In our experiments, we found the Alphasense calibration models to perform better with normalized data. However, for sake of completeness, we present here results on both normalized and unnormalized data.

**In particular, the results with normalized data in part (a) below reproduce portions of Table S2 in the supplement to the accompanying paper w.r.t the dataset DD1(Jun)**. However, note that Table S2 in the paper also reports results on the dataset DM2(Oct) which are not reproduced below.

#### (a) With normalised data

In [11]:
algoList = [ 'AS1', 'AS2', 'AS3', 'AS4' ]

p.doExperiment( g1, algoList, datasets, datasets_norm, ASNorm = True )


--------------------------------------------------------------
Training for split: 0

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 1

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 2

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 3

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 4

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 5

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

---

In [12]:
p.gen_errorMetrics( g1, algoList, datasets, datasets_norm )

# Generating mean and std of error metrics over all the splits
p.gen_avgErrorMetrics( g1, algoList, datasets, datasets_norm )

In [13]:
print('Results for AS3 with normalised data') 
algo = 'AS3'

res_o3 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'o3' + '.csv', index_col = 0 )
res_no2 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'no2' + '.csv', index_col = 0 )

res = pd.merge(res_o3, res_no2, left_index = True, right_index = True, suffixes=('_O3', '_NO2') )
res

Results for AS3 with normalised data


,MAE_O3,RMSE_O3,MAPE(%)_O3,R2_O3,MAE_NO2,RMSE_NO2,MAPE(%)_NO2,R2_NO2
DD1(Jun).csv,251.78±0.27,253.56±0.27,1341.91±120.93,-178.683±3.94,107.7±0.11,108.24±0.12,1954.6±51.77,-95.544±3.504
DD1(Oct).csv,260.39±0.36,261.42±0.34,1473.1±28.05,-58.621±0.642,124.01±0.32,126.08±0.36,780.62±9.84,-28.252±0.678


#### (b) With un-normalised data

In [14]:
algoList = [ 'AS1', 'AS2', 'AS3', 'AS4' ]

p.doExperiment( g1, algoList, datasets, datasets_norm, ASNorm = False )


--------------------------------------------------------------
Training for split: 0

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 1

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 2

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 3

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 4

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

--------------------------------------------------------------
Training for split: 5

Dataset: DD1(Jun).csv
AS1 AS2 AS3 AS4 

Dataset: DD1(Oct).csv
AS1 AS2 AS3 AS4 

---

In [15]:
p.gen_errorMetrics( g1, algoList, datasets, datasets_norm )

# Generating mean and std of error metrics over all the splits
p.gen_avgErrorMetrics( g1, algoList, datasets, datasets_norm )

In [16]:
print('Results for AS3 with un-normalised data')
algo = 'AS3'

res_o3 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'o3' + '.csv', index_col = 0 )
res_no2 = pd.read_csv( g1.errorMetricsPath + algo + '_mean_std_' + 'no2' + '.csv', index_col = 0 )

res = pd.merge(res_o3, res_no2, left_index = True, right_index = True, suffixes=('_O3', '_NO2') )
res

Results for AS3 with un-normalised data


,MAE_O3,RMSE_O3,MAPE(%)_O3,R2_O3,MAE_NO2,RMSE_NO2,MAPE(%)_NO2,R2_NO2
DD1(Jun).csv,1044.68±8.31,1416.45±8.9,5442.92±955.24,-5606.108±130.222,313.55±1.98,388.29±2.15,6643.62±300.03,-1241.3±42.954
DD1(Oct).csv,429.65±5.51,610.59±9.39,1828.26±31.53,-324.269±9.118,151.82±1.29,184.28±2.08,1413.03±40.41,-61.517±2.521


# Experiment 2: calibration results on derived datasets

### Generate temporally averaged datasets

In [17]:
import gen_derivedDataset as dd

# List of different (rolling) window sizes over which averaging is to be done
windowList = [ 5, 15, 30, 60 ]

datasetList = [ 'DD1(Jun).csv' ]

for dataset in datasetList:
    for window in windowList:
        dd.avg_dataset( g1, dataset, window = window, offset = 0 )

### Generate sub-sampled datasets

In [18]:
for dataset in datasetList:
    dd.small_dataset( g1, dataset )

### Generate aggregated datasets

In [19]:
datasetList = [ 'DD1' ]

for dataset in datasetList:
    dd.agg_dataset( g1, dataset )

In [20]:
g2 = globals.My_Globals()
g2.datasetPath = g2.derivedDatasetPath
g2.datasetList = os.listdir( g2.datasetPath )
g2.datasetList.sort()

### Load a dataset and perform preprocessing
**Note**: The calibration algorithms rely on train-test splits to obtain an unbiased estimate of the performance of various calibration algorithms. If you wish to reproduce the results in the paper, please use the train-test supplied for the 2 datasets DD1(Jun) and DD1(Oct) in the directory [perm/](perm/).

To do so, please use ```new_permutations = False``` below. Using ```new_permutations = True``` instead, allows fresh train-test splits to be created which will then overwrite the supplied train-test splits. However, minor differences are to be expected in the results if fresh splits are used to conduct the experiments.

In [21]:
datasets, datasets_norm = p.load_data( g2, new_permutations = False )

In [22]:
# Displaying some top rows of the training data for the dataset DD1(Jun) for split 0  
datasets_norm[0]['DD1(Jun)-AVG5.csv']['train'].head()

,Ref. O3(ppb),Ref. NO2(ppb),Temp(C),RH(%),no2op1(mV),no2op2(mV),o3op1(mV),o3op2(mV),no2op1 - no2op2(mV),o3op1 - o3op2(mV)
3429,33.6262,12.357400,0.188662,0.533762,0.070043,0.034459,0.062412,0.035154,0.335394,0.188905
8633,37.5220,2.450400,-0.285480,0.533762,-1.016459,-1.006539,-0.968380,-1.011073,-0.435960,-0.206644
967,32.6926,12.741800,1.368504,-2.457248,1.365363,1.417936,1.361328,1.404907,0.009052,0.386680
7704,34.5014,9.748000,-0.384719,0.533762,0.853108,0.939822,0.762515,0.968711,-0.465627,-0.847058
5348,19.4820,6.595333,-1.228250,0.533762,0.030890,-0.006231,-0.092505,0.009401,0.335394,-0.639865


### Train models for various calibration algorithms

In [23]:
algoList = [ 'LS', 'NW(ML)', 'KNN-D(ML)' ]

p.doExperiment( g2, algoList, datasets, datasets_norm )


--------------------------------------------------------------
Training for split: 0

Dataset: DD1(Jun)-AVG15.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG30.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG5.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG60.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-SMALL.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun-Oct).csv
LS NW(ML) KNN-D(ML) 

--------------------------------------------------------------
Training for split: 1

Dataset: DD1(Jun)-AVG15.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG30.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG5.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG60.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-SMALL.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun-Oct).csv
LS NW(ML) KNN-D(ML) 

--------------------------------------------------------------
Training for split: 2

Dataset: DD1(Jun)-AVG15.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG30.csv
LS NW(ML) KNN-D(ML) 

Dataset: DD1(Jun)-AVG5.csv
LS NW(ML) KNN-D(ML) 

### Calculate MAE, RMSE, MAPE(%), R2 values

In [24]:
g2.errorMetricsPath = g2.derivedErrorMetricsPath
p.gen_errorMetrics( g2, algoList, datasets, datasets_norm )

# Generating mean and std of error metrics over all the splits
p.gen_avgErrorMetrics( g2, algoList, datasets, datasets_norm )

In [25]:
# Results for KNN-D(ML) and LS for O3 corresponding to Table 6 in the paper
algo1, algo2 = 'KNN-D(ML)', 'LS'
gas = 'o3'

res1 = pd.read_csv( g2.derivedErrorMetricsPath + algo1 + '_mean_std_' + gas + '.csv', index_col = 0 )
res2 = pd.read_csv( g2.derivedErrorMetricsPath + algo2 + '_mean_std_' + gas + '.csv', index_col = 0 )
res_o3_t6 = pd.merge(res1, res2, left_index = True, right_index = True, suffixes=('_KNN-D(ML)', '_LS') )

In [26]:
# Results for KNN-D(ML) and LS for NO2 corresponding to Table 6 in the paper
algo1, algo2 = 'KNN-D(ML)', 'LS'
gas = 'no2'

res1 = pd.read_csv( g2.derivedErrorMetricsPath + algo1 + '_mean_std_' + gas + '.csv', index_col = 0 )
res2 = pd.read_csv( g2.derivedErrorMetricsPath + algo2 + '_mean_std_' + gas + '.csv', index_col = 0 )
res_no2_t6 = pd.merge(res1, res2, left_index = True, right_index = True, suffixes=('_KNN-D(ML)', '_LS') )

# Experiment 3: Transfer experiments

In [27]:
# Initialising Global Variables. For details, refer to globals.py
g3 = globals.My_Globals()

### Load a dataset and perform preprocessing
**Note**: The calibration algorithms rely on train-test splits to obtain an unbiased estimate of the performance of various calibration algorithms. If you wish to reproduce the results in the paper, please use the train-test supplied for the 2 datasets DD1(Jun) and DD1(Oct) in the directory [perm/](perm/).

To do so, please use ```new_permutations = False``` below. Using ```new_permutations = True``` instead, allows fresh train-test splits to be created which will then overwrite the supplied train-test splits. However, minor differences are to be expected in the results if fresh splits are used to conduct the experiments.

In [28]:
datasets, datasets_norm = p.load_data( g3, new_permutations = False )

In [29]:
# Displaying some top rows of the testing data for the dataset DD1(Jun) for split 0  
datasets[0]['DD1(Jun).csv']['test'].head()

,Ref. O3(ppb),Ref. NO2(ppb),Temp(C),RH(%),no2op1(mV),no2op2(mV),o3op1(mV),o3op2(mV),no2op1 - no2op2(mV),o3op1 - o3op2(mV)
14385,101.743,9.919,35.5,99.9,134.0,134.0,184.0,159.0,0.0,25.0
30727,9.865,38.000,27.6,99.9,84.0,61.0,95.0,83.0,23.0,12.0
45673,21.437,2.306,28.0,99.9,158.0,153.0,167.0,180.0,5.0,-13.0
42475,21.127,5.662,26.7,99.9,89.0,84.0,99.0,106.0,5.0,-7.0
12073,19.567,8.581,28.1,99.9,192.0,176.0,209.0,201.0,16.0,8.0


### Test models by attempting to transfer them to a different dataset

In [30]:
algoList = ['LS','KNN-D(ML)']
datasetTrainList = ['DD1(Jun).csv']
datasetTestList  = ['DD1(Oct).csv']
datasetList = [ datasetTrain[:-4] + ' -> ' + datasetTest[:-4] for datasetTrain, datasetTest in zip( datasetTrainList, datasetTestList ) ]

p.transferTesting( g3, algoList, datasetTrainList, datasetTestList, datasets, datasets_norm )


--------------------------------------------------------------
Testing for split: 0

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split: 1

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split: 2

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split: 3

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split: 4

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split: 5

trainDataset: DD1(Jun).csv testDataset: DD1(Oct).csv
LS KNN-D(ML) 

--------------------------------------------------------------
Testing for split:

### Calculate MAE, RMSE, MAPE(%), R2 values

In [31]:
p.gen_transferErrorMetrics( g3, algoList, datasetTrainList, datasetTestList, datasetList, datasets, datasets_norm )

g3.errorMetricsPath = g3.transferErrorMetricsPath
g3.datasetList = datasetList
# Generating mean and std of error metrics over all the splits
p.gen_avgErrorMetrics( g3, algoList, datasets, datasets_norm )

In [32]:
# Results for KNN-D(ML) and LS for O3 corresponding to Table 6 in the paper
algo1, algo2 = 'KNN-D(ML)', 'LS'
gas = 'o3'

res1 = pd.read_csv( g3.transferErrorMetricsPath + algo1 + '_mean_std_' + gas + '.csv', index_col = 0 )
res2 = pd.read_csv( g3.transferErrorMetricsPath + algo2 + '_mean_std_' + gas + '.csv', index_col = 0 )
res = pd.merge(res1, res2, left_index = True, right_index = True, suffixes=('_KNN-D(ML)', '_LS') )
res_o3_t6 = res.append(res_o3_t6)

In [33]:
# Results for KNN-D(ML) and LS for NO2 corresponding to Table 6 in the paper
algo1, algo2 = 'KNN-D(ML)', 'LS'
gas = 'no2'

res1 = pd.read_csv( g3.transferErrorMetricsPath + algo1 + '_mean_std_' + gas + '.csv', index_col = 0 )
res2 = pd.read_csv( g3.transferErrorMetricsPath + algo2 + '_mean_std_' + gas + '.csv', index_col = 0 )
res = pd.merge(res1, res2, left_index = True, right_index = True, suffixes=('_KNN-D(ML)', '_LS') )
res_no2_t6 = res.append(res_no2_t6)

# Final Results

### 1. MAE and R2 for KNN-D(ML) averaged over 10 splits.
**This reproduces portions of Table 4 in the accompanying paper w.r.t the dataset DD1(Jun)**. Note that Table 4 in the paper also reports results on the MM5(Jun) and (Oct) datasets which are not reproduced below. Also, Table 4 in the paper does not report MAE values which are reported below.

Upon executing the code block below, the following table should be printed as output if code execution has been performed correctly.
#### <div align="center">Expected Results for KNN-D(ML)</div>

||MAE_O3|R2_O3|MAE_NO2|R2_NO2|
|---|---|---|---|---|
|__DD1(Jun).csv__|3.52 ± 0.03|0.923 ± 0.003|2.67 ± 0.05|0.819 ± 0.015|
|__DD1(Oct).csv__|2.1 ± 0.06|0.99 ± 0.001|2.16 ± 0.05|0.977 ± 0.002|

In [34]:
res_t4[['MAE_O3', 'R2_O3', 'MAE_NO2', 'R2_NO2']].loc[['DD1(Jun).csv', 'DD1(Oct).csv']]

,MAE_O3,R2_O3,MAE_NO2,R2_NO2
DD1(Jun).csv,3.52±0.03,0.923±0.003,2.67±0.05,0.819±0.015
DD1(Oct).csv,2.1±0.06,0.99±0.001,2.16±0.05,0.977±0.002


### 2. Win Matrix for O<sub>3</sub> averaged over 2 datasets
**Please note that although this corresponds to the experiment reported by Table 3 in the accompanying paper, those results are averaged over 12 datasets and cannot be reproduced by the results below that are averaged over only 2 datasets.**

In [35]:
winMatrix_o3[['LS', 'RT', 'KRR', 'NW(ML)', 'KNN-D(ML)']].loc[['LS', 'RT', 'KRR', 'NW(ML)', 'KNN-D(ML)']]

,LS,RT,KRR,NW(ML),KNN-D(ML)
LS,0.0,0.0,0.00,0.00,0.0
RT,1.0,0.0,0.45,0.00,0.0
KRR,1.0,0.5,0.00,0.05,0.0
NW(ML),1.0,1.0,0.80,0.00,0.0
KNN-D(ML),1.0,1.0,1.00,1.00,0.0


### 3. Win Matrix for NO<sub>2</sub> averaged over 2 datasets
**Please note that although this corresponds to the experiment reported by Table 3 in the accompanying paper, those results are averaged over 12 datasets and cannot be reproduced by the results below that are averaged over only 2 datasets.**

In [36]:
winMatrix_no2[['LS', 'RT', 'KRR', 'NW(ML)', 'KNN-D(ML)']].loc[['LS', 'RT', 'KRR', 'NW(ML)', 'KNN-D(ML)']]

,LS,RT,KRR,NW(ML),KNN-D(ML)
LS,0.0,0.0,0.0,0.00,0.0
RT,1.0,0.0,1.0,0.35,0.0
KRR,1.0,0.0,0.0,0.00,0.0
NW(ML),1.0,0.5,1.0,0.00,0.0
KNN-D(ML),1.0,1.0,1.0,1.00,0.0


### 4. MAE and R2 for O<sub>3</sub> in transfer experiments
**This reproduces portions of Table 6 in the accompanying paper w.r.t the transfer experiments DD1(Jun) -> (Oct) and the dataset DD1(Jun-Oct) for the gas O3**. However, note that Table 6 in the paper also reports results on several other transfers e.g. MM5(Oct) -> (Jun) which are not reproduced below.

Upon executing the code block below, the following table should be printed as output if code execution has been performed correctly.
#### <div align="center">Expected Results for KNN-D(ML) and LS for O<sub>3</sub></div>

||MAE_KNN-D(ML)|R2_KNN-D(ML)|MAE_LS|R2_LS|
|---|---|---|---|---|
|__DD1(Jun) -> DD1(Oct)__|21.7 ± 2.57|0.193 ± 0.188|12.88 ± 0.17|0.729 ± 0.004|
|__DD1(Jun-Oct).csv__|3.29 ± 0.07|0.956 ± 0.002|7.17 ± 0.02|0.842 ± 0.002|

In [37]:
res_o3_t6[['MAE_KNN-D(ML)', 'R2_KNN-D(ML)', 'MAE_LS', 'R2_LS']].loc[['DD1(Jun) -> DD1(Oct)','DD1(Jun-Oct).csv']]

,MAE_KNN-D(ML),R2_KNN-D(ML),MAE_LS,R2_LS
DD1(Jun) -> DD1(Oct),21.7±2.57,0.193±0.188,12.88±0.17,0.729±0.004
DD1(Jun-Oct).csv,3.29±0.07,0.956±0.002,7.17±0.02,0.842±0.002


### 5. MAE and R2 for NO<sub>2</sub> in transfer experiments
**This reproduces portions of Table 6 in the accompanying paper w.r.t the transfer experiments DD1(Jun) -> (Oct) and the dataset DD1(Jun-Oct) for the gas NO2**. However, note that Table 6 in the paper also reports results on several other transfers e.g. MM5(Oct) -> (Jun) which are not reproduced below.

Upon executing the code block below, the following table should be printed as output if code execution has been performed correctly.
#### <div align="center">Expected Results for KNN-D(ML) and LS for NO<sub>2</sub></div>

||MAE_KNN-D(ML)|R2_KNN-D(ML)|MAE_LS|R2_LS|
|---|---|---|---|---|
|__DD1(Jun) -> DD1(Oct)__|21.86 ± 2.07|-0.642 ± 0.268|12.73 ± 0.2|0.219 ± 0.007| 
|__DD1(Jun-Oct).csv__|2.62 ± 0.07|0.924 ± 0.003|8.16 ± 0.03|0.482 ± 0.007| 

In [38]:
res_no2_t6[['MAE_KNN-D(ML)', 'R2_KNN-D(ML)', 'MAE_LS', 'R2_LS']].loc[['DD1(Jun) -> DD1(Oct)','DD1(Jun-Oct).csv']]

,MAE_KNN-D(ML),R2_KNN-D(ML),MAE_LS,R2_LS
DD1(Jun) -> DD1(Oct),21.86±2.07,-0.642±0.268,12.73±0.2,0.219±0.007
DD1(Jun-Oct).csv,2.62±0.07,0.924±0.003,8.16±0.03,0.482±0.007
